In [1]:
import rasterio

In [2]:
import pandas as pd
import geopandas as gpd
import datetime

In [3]:
pavel = pd.read_csv('../table-data.csv')

### Vereinfachte Geometriedaten der Lankreise

In [4]:
geometry_lk_simple = pd.read_pickle("../geometry_landkreise_simplified/geometry_lk_simplified_0_001.gzip")

### Pavelstabelle mit geometrie Daten anreichern

In [5]:
merged = gpd.GeoDataFrame(pavel.join(geometry_lk_simple, on='Landkreis'))

### Nur ein Teil der Spalten übernehmen

In [7]:
subset = merged[['Landkreis', 'Bundesland', 'Einwohner', 'AnzahlFallNeu', 'AnzahlFallNeu-7-Tage-Dropped', 'AnzahlTodesfallNeu-7-Tage', 'Fallsterblichkeit-Prozent', 'Kontaktrisiko', 'InzidenzFallNeu-7-Tage', 'MeldeDauerFallNeu-Schnitt', 'InzidenzFallNeu-7-Tage-Trend-Spezial', 'geometry']]
subset['Kehrwert_risiko'] = 1/subset['Kontaktrisiko']  # zur vereinfachten Farbzuweisung in der Karte 

/Users/bjoern/Working/covid19/Corona/venv/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### Entfernen der Bundesländer/Republik Zeilen

In [8]:
subset = subset[subset.geometry != None]

In [9]:
subset

,Landkreis,Bundesland,Einwohner,AnzahlFallNeu,AnzahlFallNeu-7-Tage-Dropped,AnzahlTodesfallNeu-7-Tage,Fallsterblichkeit-Prozent,Kontaktrisiko,InzidenzFallNeu-7-Tage,MeldeDauerFallNeu-Schnitt,InzidenzFallNeu-7-Tage-Trend-Spezial,geometry,Kehrwert_risiko
0,LK Greiz,Thüringen,97398,97,1,1,2.922856,44.862639,386.044888,0.777778,1.649717,"MULTIPOLYGON (((12.19167 50.96358, 12.18251 50...",0.022290
1,LK Peine,Niedersachsen,134801,70,0,1,2.534562,61.759740,178.782057,0.531250,2.587632,"MULTIPOLYGON (((10.29191 52.44790, 10.25195 52...",0.016192
2,SK Ansbach,Bayern,41798,8,0,1,3.094340,62.739106,133.977702,0.428571,3.399078,"MULTIPOLYGON (((10.58451 49.34231, 10.57116 49...",0.015939
3,SK Hof,Bayern,45825,42,0,4,3.024352,77.635148,342.607747,0.000000,1.074178,"MULTIPOLYGON (((11.88020 50.35407, 11.87482 50...",0.012881
4,LK Ansbach,Bayern,184591,26,23,12,2.811550,79.731357,139.768461,7.631579,2.563856,"MULTIPOLYGON (((10.22175 49.49031, 10.11833 49...",0.012542
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,LK Northeim,Niedersachsen,132285,6,0,3,2.087287,2435.206438,24.190195,0.500000,0.485017,"MULTIPOLYGON (((9.66096 51.56253, 9.67238 51.5...",0.000411
425,LK Cochem-Zell,Rheinland-Pfalz,61375,1,0,1,3.008963,2692.030937,21.181263,0.000000,0.501072,"MULTIPOLYGON (((7.07428 50.27758, 7.06878 50.2...",0.000371
426,SK Speyer,Rheinland-Pfalz,50561,1,0,2,3.938946,3415.169572,17.800281,0.000000,0.469995,"MULTIPOLYGON (((8.42361 49.36754, 8.40676 49.3...",0.000293
427,LK Dithmarschen,Schleswig-Holstein,133193,5,0,0,2.958580,3474.054960,11.261853,0.250000,0.730273,"MULTIPOLYGON (((8.69258 54.06045, 8.69844 54.0...",0.000288


In [11]:
now = datetime.datetime.now()
subset.to_file("../leaflet-website/risk_{:%Y-%m-%d}.geojson".format(now), driver='GeoJSON')